==== UTS Data Preparation ====== 

1. Data Cleaning

In [85]:

# => Load Dataset
import pandas as pd
import numpy as np

df = pd.read_csv("user_behavior_raw.csv")
df.head()


,User ID,Device Model,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
0,1.0,Google Pixel 5,Android,393.0,6.4,1872.0,67,1122,NaN,Male,4
1,2.0,OnePlus 9,NaN,268.0,4.7,1331.0,42,944,4778.0,Female,3
2,3.0,Xiaomi Mi 11,Android,154.0,4.0,761.0,32,322,42.0,Male,2
3,4.0,Google Pixel 5,Android,239.0,4.8,1676.0,56,871,20.0,Male,3
4,5.0,iPhone 12,iOS,187.0,4.3,1367.0,58,988,31.0,Female,3


In [86]:
# => Cek Struktur Data
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User ID                     687 non-null    float64
 1   Device Model                690 non-null    object 
 2   Operating System            688 non-null    object 
 3   App Usage Time (min/day)    689 non-null    float64
 4   Screen On Time (hours/day)  688 non-null    float64
 5   Battery Drain (mAh/day)     719 non-null    float64
 6   Number of Apps Installed    720 non-null    int64  
 7   Data Usage (MB/day)         720 non-null    int64  
 8   Age                         719 non-null    float64
 9   Gender                      720 non-null    object 
 10  User Behavior Class         720 non-null    int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 62.0+ KB


In [87]:
# => Cek Missin Value
df.isnull().sum()

User ID                       33
Device Model                  30
Operating System              32
App Usage Time (min/day)      31
Screen On Time (hours/day)    32
Battery Drain (mAh/day)        1
Number of Apps Installed       0
Data Usage (MB/day)            0
Age                            1
Gender                         0
User Behavior Class            0
dtype: int64

In [88]:
# => Cek Duplikasi
duplicates = df.duplicated().sum()
print("Jumlah duplikasi:", duplicates)

df = df.drop_duplicates()



Jumlah duplikasi: 18


In [89]:
# => Standardisasi Nama Kolom
df.columns = [c.strip().lower().replace(" ", "_").replace("-", "_") for c in df.columns]
df.head()


,user_id,device_model,operating_system,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,gender,user_behavior_class
0,1.0,Google Pixel 5,Android,393.0,6.4,1872.0,67,1122,NaN,Male,4
1,2.0,OnePlus 9,NaN,268.0,4.7,1331.0,42,944,4778.0,Female,3
2,3.0,Xiaomi Mi 11,Android,154.0,4.0,761.0,32,322,42.0,Male,2
3,4.0,Google Pixel 5,Android,239.0,4.8,1676.0,56,871,20.0,Male,3
4,5.0,iPhone 12,iOS,187.0,4.3,1367.0,58,988,31.0,Female,3


In [90]:
# => Tangani Missing Values
num_cols = df.select_dtypes(include=['int64','float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# isi missing numerik dengan median
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# isi missing kategorikal dengan mode
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

df.isnull().sum()


user_id                       0
device_model                  0
operating_system              0
app_usage_time_(min/day)      0
screen_on_time_(hours/day)    0
battery_drain_(mah/day)       0
number_of_apps_installed      0
data_usage_(mb/day)           0
age                           0
gender                        0
user_behavior_class           0
dtype: int64

In [91]:
# => Deteksi Outlier (IQR Method)
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    print(col, "→", outliers, "outliers")


user_id → 4 outliers
app_usage_time_(min/day) → 4 outliers
screen_on_time_(hours/day) → 5 outliers
battery_drain_(mah/day) → 5 outliers
number_of_apps_installed → 5 outliers
data_usage_(mb/day) → 5 outliers
age → 6 outliers
user_behavior_class → 5 outliers


In [92]:
# => Tangani Outlier (Capping)
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower, upper)

df.head()


,user_id,device_model,operating_system,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,gender,user_behavior_class
0,1.0,Google Pixel 5,Android,393.0,6.4,1872.0,67.0,1122.0,39.0,Male,4
1,2.0,OnePlus 9,Android,268.0,4.7,1331.0,42.0,944.0,83.0,Female,3
2,3.0,Xiaomi Mi 11,Android,154.0,4.0,761.0,32.0,322.0,42.0,Male,2
3,4.0,Google Pixel 5,Android,239.0,4.8,1676.0,56.0,871.0,20.0,Male,3
4,5.0,iPhone 12,iOS,187.0,4.3,1367.0,58.0,988.0,31.0,Female,3


2. Data Integration


In [93]:
import pandas as pd

# df1 = pd.read_csv("user_behavior_dataset.csv")         # Dataset utama
df2 = pd.read_csv("user_demographic_dummy.csv")        # Dataset kedua

df.head(), df2.head()


(   user_id    device_model operating_system  app_usage_time_(min/day)  \
 0      1.0  Google Pixel 5          Android                     393.0   
 1      2.0       OnePlus 9          Android                     268.0   
 2      3.0    Xiaomi Mi 11          Android                     154.0   
 3      4.0  Google Pixel 5          Android                     239.0   
 4      5.0       iPhone 12              iOS                     187.0   
 
    screen_on_time_(hours/day)  battery_drain_(mah/day)  \
 0                         6.4                   1872.0   
 1                         4.7                   1331.0   
 2                         4.0                    761.0   
 3                         4.8                   1676.0   
 4                         4.3                   1367.0   
 
    number_of_apps_installed  data_usage_(mb/day)   age  gender  \
 0                      67.0               1122.0  39.0    Male   
 1                      42.0                944.0  83.0  Female 

In [94]:
# Sesuaikan nama kolom untuk proses join
df.rename(columns={'User_ID': 'user_id'}, inplace=True)
df.head()


,user_id,device_model,operating_system,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,gender,user_behavior_class
0,1.0,Google Pixel 5,Android,393.0,6.4,1872.0,67.0,1122.0,39.0,Male,4
1,2.0,OnePlus 9,Android,268.0,4.7,1331.0,42.0,944.0,83.0,Female,3
2,3.0,Xiaomi Mi 11,Android,154.0,4.0,761.0,32.0,322.0,42.0,Male,2
3,4.0,Google Pixel 5,Android,239.0,4.8,1676.0,56.0,871.0,20.0,Male,3
4,5.0,iPhone 12,iOS,187.0,4.3,1367.0,58.0,988.0,31.0,Female,3


In [95]:
# Cek key apakah cocok
df['user_id'].head(), df2['user_id'].head()


(0    1.0
 1    2.0
 2    3.0
 3    4.0
 4    5.0
 Name: user_id, dtype: float64,
 0    1
 1    2
 2    3
 3    4
 4    5
 Name: user_id, dtype: int64)

In [96]:
# Lakukan proses Data Integration (Merge)
df_merged = pd.merge(df, df2, on='user_id', how='left')
df_merged.head()


/tmp/ipykernel_6094/1084157003.py:2: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_merged = pd.merge(df, df2, on='user_id', how='left')


,user_id,device_model,operating_system,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,gender,user_behavior_class,age_group,region
0,1.0,Google Pixel 5,Android,393.0,6.4,1872.0,67.0,1122.0,39.0,Male,4,18-25,Asia
1,2.0,OnePlus 9,Android,268.0,4.7,1331.0,42.0,944.0,83.0,Female,3,26-35,Europe
2,3.0,Xiaomi Mi 11,Android,154.0,4.0,761.0,32.0,322.0,42.0,Male,2,36-45,America
3,4.0,Google Pixel 5,Android,239.0,4.8,1676.0,56.0,871.0,20.0,Male,3,18-25,Asia
4,5.0,iPhone 12,iOS,187.0,4.3,1367.0,58.0,988.0,31.0,Female,3,26-35,Europe


In [97]:
# cek hasil integration
df_merged.info()
# df_merged[['user_id', 'age_group', 'region']].head(10)     => cek valid


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     702 non-null    float64
 1   device_model                702 non-null    object 
 2   operating_system            702 non-null    object 
 3   app_usage_time_(min/day)    702 non-null    float64
 4   screen_on_time_(hours/day)  702 non-null    float64
 5   battery_drain_(mah/day)     702 non-null    float64
 6   number_of_apps_installed    702 non-null    float64
 7   data_usage_(mb/day)         702 non-null    float64
 8   age                         702 non-null    float64
 9   gender                      702 non-null    object 
 10  user_behavior_class         702 non-null    int64  
 11  age_group                   483 non-null    object 
 12  region                      483 non-null    object 
dtypes: float64(7), int64(1), object(5)


3. Data Transformtion

In [98]:
df = df_merged
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     702 non-null    float64
 1   device_model                702 non-null    object 
 2   operating_system            702 non-null    object 
 3   app_usage_time_(min/day)    702 non-null    float64
 4   screen_on_time_(hours/day)  702 non-null    float64
 5   battery_drain_(mah/day)     702 non-null    float64
 6   number_of_apps_installed    702 non-null    float64
 7   data_usage_(mb/day)         702 non-null    float64
 8   age                         702 non-null    float64
 9   gender                      702 non-null    object 
 10  user_behavior_class         702 non-null    int64  
 11  age_group                   483 non-null    object 
 12  region                      483 non-null    object 
dtypes: float64(7), int64(1), object(5)


In [99]:
# Encoding Data Kategorikal
df = pd.get_dummies(df, columns=['gender', 'operating_system', 'age_group', 'region'], drop_first=True)
df.head()


,user_id,device_model,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,user_behavior_class,gender_Female,gender_Male,gender_female,operating_system_iOS,age_group_26-35,age_group_36-45,region_Asia,region_Europe
0,1.0,Google Pixel 5,393.0,6.4,1872.0,67.0,1122.0,39.0,4,False,True,False,False,False,False,True,False
1,2.0,OnePlus 9,268.0,4.7,1331.0,42.0,944.0,83.0,3,True,False,False,False,True,False,False,True
2,3.0,Xiaomi Mi 11,154.0,4.0,761.0,32.0,322.0,42.0,2,False,True,False,False,False,True,False,False
3,4.0,Google Pixel 5,239.0,4.8,1676.0,56.0,871.0,20.0,3,False,True,False,False,False,False,True,False
4,5.0,iPhone 12,187.0,4.3,1367.0,58.0,988.0,31.0,3,True,False,False,True,True,False,False,True


In [100]:
# Scaling Data Numerik
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

num_cols = [
    'screen_on_time_(hours/day)', 
    'battery_drain_(mah/day)', 
    'app_usage_time_(min/day)', 
    
    ]

df[num_cols] = scaler.fit_transform(df[num_cols])

df.head()


,user_id,device_model,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,user_behavior_class,gender_Female,gender_Male,gender_female,operating_system_iOS,age_group_26-35,age_group_36-45,region_Asia,region_Europe
0,1.0,Google Pixel 5,0.424313,0.403361,0.368048,67.0,1122.0,39.0,4,False,True,False,False,False,False,True,False
1,2.0,OnePlus 9,0.278200,0.276377,0.241224,42.0,944.0,83.0,3,True,False,False,False,True,False,False,True
2,3.0,Xiaomi Mi 11,0.144944,0.224090,0.107601,32.0,322.0,42.0,2,False,True,False,False,False,True,False,False
3,4.0,Google Pixel 5,0.244302,0.283847,0.322100,56.0,871.0,20.0,3,False,True,False,False,False,False,True,False
4,5.0,iPhone 12,0.183518,0.246499,0.249663,58.0,988.0,31.0,3,True,False,False,True,True,False,False,True


4. Data Reduction

In [103]:
# Pisahkan kolom numerik saja untuk PCA
num_df = df.select_dtypes(include=['int64', 'float64'])
num_df.head()

,user_id,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,user_behavior_class
0,1.0,0.424313,0.403361,0.368048,67.0,1122.0,39.0,4
1,2.0,0.278200,0.276377,0.241224,42.0,944.0,83.0,3
2,3.0,0.144944,0.224090,0.107601,32.0,322.0,42.0,2
3,4.0,0.244302,0.283847,0.322100,56.0,871.0,20.0,3
4,5.0,0.183518,0.246499,0.249663,58.0,988.0,31.0,3


In [ ]:


# Standarisasi Data (wajib sebelum PCA)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_scaled = scaler.fit_transform(num_df)


In [ ]:
# Lakukan PCA 
# Dimensionality Reduction
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_result = pca.fit_transform(num_scaled)


In [ ]:
# Masukkan hasil PCA ke dataframe baru
df_pca = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
df_pca.head()


,PC1,PC2
0,1.203639,-1.219249
1,-0.364924,1.179502
2,-1.768541,-1.084282
3,-0.035838,-2.274962
4,-0.275048,-1.669605


In [ ]:
# Gabungkan dengan kolom target (jika ada)
if 'Behavior_Class' in df.columns:
    df_pca['Behavior_Class'] = df['Behavior_Class']


In [ ]:
# Cek variansi yang dijelaskan oleh PCA
pca.explained_variance_ratio_


array([0.66061729, 0.12694286])

=> PC1 menyimpan 72% informasi 
=> PC2 menyimpan 15% informasi
Total 87% informasi sudah cukup bagus

In [ ]:
# kode ini bisa disalin untuk mencetak implementasi sebelumnya
# df_pca.to_csv("user_behavior_reduced.csv", index=False)


In [ ]:
# FEATURE SELECTION
#  A. Implementasi: Variance Threshold
from sklearn.feature_selection import VarianceThreshold

df_num = df.select_dtypes(include=['int64','float64'])

selector = VarianceThreshold(threshold=0.01)
reduced_features = selector.fit_transform(df_num)

df_feature_selected = pd.DataFrame(
    reduced_features,
    columns=df_num.columns[selector.get_support()]
)

df_feature_selected.head()
 



,user_id,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,user_behavior_class
0,1.0,0.424313,0.403361,0.368048,67.0,1122.0,39.0,4.0
1,2.0,0.278200,0.276377,0.241224,42.0,944.0,83.0,3.0
2,3.0,0.144944,0.224090,0.107601,32.0,322.0,42.0,2.0
3,4.0,0.244302,0.283847,0.322100,56.0,871.0,20.0,3.0
4,5.0,0.183518,0.246499,0.249663,58.0,988.0,31.0,3.0


In [ ]:
# B. Implementasi: SelectKBest (ANOVA) 
# from sklearn.feature_selection import SelectKBest, f_classif

# X = df_num
# y = df['Behavior_Class']   # jika ada kolom target

# selector = SelectKBest(score_func=f_classif, k=5)
# X_selected = selector.fit_transform(X, y)

# selected_columns = X.columns[selector.get_support()]
# selected_columns


In [ ]:
# NUMEROSITY REDUCTION
# A. Random Sampling (mengurangi jumlah baris)
df_sample = df.sample(frac=0.3, random_state=42)
df_sample.head()


,user_id,device_model,app_usage_time_(min/day),screen_on_time_(hours/day),battery_drain_(mah/day),number_of_apps_installed,data_usage_(mb/day),age,user_behavior_class,gender_Female,gender_Male,gender_female,operating_system_iOS,age_group_26-35,age_group_36-45,region_Asia,region_Europe
493,494.0,OnePlus 9,0.349503,0.433240,0.372736,69.0,1383.0,23.0,4,False,True,False,False,False,False,True,False
164,165.0,Xiaomi Mi 11,0.289889,0.283847,0.219422,48.0,851.0,43.0,3,True,False,False,False,True,False,False,True
54,55.0,Google Pixel 5,0.568089,0.575163,0.598957,83.0,2189.0,53.0,5,True,False,False,False,True,False,False,True
641,642.0,OnePlus 9,0.023378,0.029879,0.033054,16.0,255.0,26.0,1,True,False,False,False,False,False,False,False
607,608.0,Xiaomi Mi 11,0.202221,0.298786,0.239348,56.0,787.0,21.0,3,False,True,False,False,False,False,False,False


In [ ]:
# cari kolom dummy (Menambal dataset yang sudah di-One-Hot Encoding)
region_cols = [c for c in df.columns if c.startswith("region_")]

# buat kolom region berdasarkan nama dummy yang nilainya 1
df['region'] = df[region_cols].idxmax(axis=1).str.replace("region_", "")

# B. Aggregation (meringkas data)

df_region = df.groupby('region').agg({
    'screen_on_time_(hours/day)': 'mean',
    'battery_drain_(mah/day)': 'mean',
    'app_usage_time_(min/day)': 'mean'
}).reset_index()

df_region




,region,screen_on_time_(hours/day),battery_drain_(mah/day),app_usage_time_(min/day)
0,Asia,0.324325,0.291782,0.280308
1,Europe,0.327166,0.292137,0.296727
